# Time Series Generation with Genesis

Generate synthetic time series data while preserving temporal patterns.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from genesis.generators.timeseries import StatisticalTimeSeriesGenerator, TimeGANGenerator

## Create Sample Time Series

In [ ]:
np.random.seed(42)
n = 500

t = np.arange(n)
trend = 0.05 * t
seasonality = 10 * np.sin(2 * np.pi * t / 24)  # Daily pattern
noise = np.random.normal(0, 2, n)

data = pd.DataFrame({
    'temperature': 20 + trend + seasonality + noise,
    'humidity': 50 + 5 * np.sin(2 * np.pi * t / 24 + np.pi/4) + np.random.normal(0, 3, n),
    'pressure': 1013 + np.cumsum(np.random.randn(n) * 0.5)
})

plt.figure(figsize=(15, 8))
for i, col in enumerate(data.columns):
    plt.subplot(3, 1, i+1)
    plt.plot(data[col])
    plt.title(col)
plt.tight_layout()
plt.show()

## Statistical Time Series Generator

In [ ]:
stat_generator = StatisticalTimeSeriesGenerator(verbose=True)
stat_generator.fit(data)

stat_synthetic = stat_generator.generate(n_samples=500)
stat_synthetic.head()

In [ ]:
# Compare real vs synthetic
fig, axes = plt.subplots(3, 2, figsize=(15, 10))

for i, col in enumerate(data.columns):
    axes[i, 0].plot(data[col], label='Real')
    axes[i, 0].set_title(f'{col} - Real')
    
    axes[i, 1].plot(stat_synthetic[col], label='Synthetic', color='orange')
    axes[i, 1].set_title(f'{col} - Synthetic')

plt.tight_layout()
plt.show()

## TimeGAN Generator

Deep learning approach for complex temporal patterns.

In [ ]:
# Prepare data in sequences
timegan_generator = TimeGANGenerator(
    seq_len=24,  # Sequence length
    n_epochs=50,  # Reduced for demo
    verbose=True
)

timegan_generator.fit(data)
timegan_synthetic = timegan_generator.generate(n_samples=500)

In [ ]:
# Compare autocorrelations
from pandas.plotting import autocorrelation_plot

fig, axes = plt.subplots(3, 2, figsize=(15, 10))

for i, col in enumerate(data.columns):
    autocorrelation_plot(data[col], ax=axes[i, 0])
    axes[i, 0].set_title(f'{col} - Real Autocorrelation')
    
    autocorrelation_plot(stat_synthetic[col], ax=axes[i, 1])
    axes[i, 1].set_title(f'{col} - Synthetic Autocorrelation')

plt.tight_layout()
plt.show()

## Time Series Evaluation

In [ ]:
from genesis.evaluation.evaluator import QualityEvaluator

evaluator = QualityEvaluator(data, stat_synthetic)
report = evaluator.evaluate()

print(report.summary())